In [1]:
import os, sys, time, random
import h5py
import numpy as np
import json

In [21]:
import pathlib
import random
import string
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization

In [52]:
text_file = pathlib.Path('kor.txt')

In [53]:
with open(text_file, encoding='UTF8') as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    eng, kor, etc = line.split("\t")
    eng = "[start] " + eng + " [end]"
    text_pairs.append((kor, eng))

In [54]:
for _ in range(5):
    print(random.choice(text_pairs))

('톰은 부모님과는 달리 프랑스어를 할 줄 몰라.', "[start] Tom doesn't speak French, but his parents do. [end]")
('톰은 자택 부엌에서 죽었어.', '[start] Tom died in his kitchen. [end]')
('무언가 바뀌었어.', '[start] Something changed. [end]')
('여기 자주 와?', '[start] Do you come here a lot? [end]')
('물러서.', '[start] Step back. [end]')


In [55]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

3853 total pairs
2699 training pairs
577 validation pairs
577 test pairs


In [56]:
from konlpy.tag import Kkma
tokenizer_kr = Kkma()

In [57]:
from nltk.tokenize import word_tokenize

In [60]:
train_kor_texts = [pair[0] for pair in train_pairs]
train_eng_texts = [pair[1] for pair in train_pairs]

In [62]:
UNKNOWN = '[UKN]'
CLOSE = '[CLS]'
PAD = '[PAD]'
START = '[SRT]'

In [65]:
def _to_word(sent: list) -> list:
    """ Filter word as stop words """
    rst = []
    for word in sent:
        word = word.strip()
        if not word or word in '"\'\\₩':
            continue
        rst.append(word)
    return rst

In [67]:
kor_tex = []
for k in train_kor_texts:
    line = k.strip()
    sents = tokenizer_kr.sentences(line)
    sents = [tokenizer_kr.morphs(sent) for sent in sents]
    sents = [_to_word(sent) for sent in sents]  
    kor_tex.append(sents)

In [68]:
kor_tex

[[['대답', '이', '직설적', '이', 'ㄴ', '것', '같', '아', '.']],
 [['톰', '은', '전', '에', '호주', '에서', '살', '았', '지', '?']],
 [['넣', 'ㄴ', '프랑스어', '를', '공부', '하', '어야', '되', '어', '.']],
 [['느', '어', '노래', '그만', '불르', '어야', '하', 'ㄹ', '것', '같', '아', '.']],
 [['낳', 'ㄴ', '이', '학교', '를', '좋아하', '어', '.']],
 [['톰', '은', '나', '보다', '돈', '이', '많', '아', '.']],
 [['얼마나', '자주', '조부모', '님', '을', '방문', '하', '니', '?']],
 [['모두', '들', '안녕', '!']],
 [['이', '사전', '은', '내', '것', '이', '다', '.']],
 [['톰', '은', '아주', '헌신적', '이', 'ㄴ', '배우', '야', '.']],
 [['톰', '은', '우리', '랑', '같이', '공원', '에', '가', '고', '싶', '지', '않', '대', '.']],
 [['톰', '이', '자', '고', '있', '는', '것', '처럼', '보이', '어', '.']],
 [['유럽', '에', '가보', 'ㄴ', '적', '있', '어', '?']],
 [['다시', '한', '번', '해보', '아', '.']],
 [['인간',
   '의',
   '몸',
   '은',
   '수많',
   '은',
   '세포',
   '들',
   '로',
   '구성',
   '되',
   '어',
   '있',
   '어',
   '.']],
 [['낳', 'ㄴ', '톰', '이랑', '테니스', '치', '어', '보', 'ㄴ', '적이', '없', '어', '.']],
 [['톰', '강아지', '는', '정말', '크', '어', '.']],
 [['도와주', '어

In [ ]:
eng_text = []


In [42]:
from tensorflow import keras
from tensorflow.keras import layers
from keras.optimizers import *
from keras.callbacks import *
from keras.models import *
from keras.layers import *
from keras.callbacks import *
import tensorflow as tf

In [43]:
import string
strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 15000
sequence_length = 20
batch_size = 64


def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


kor_vectorization = TextVectorization(
    max_tokens=None, output_mode="int", output_sequence_length=sequence_length,
)
eng_vectorization = TextVectorization(
    max_tokens=None,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_kor_texts = [pair[0] for pair in train_pairs]
train_eng_texts = [pair[1] for pair in train_pairs]
kor_vectorization.adapt(train_kor_texts)
eng_vectorization.adapt(train_eng_texts)

In [44]:
def format_dataset(kor, eng):
    kor = kor_vectorization(kor)
    eng = eng_vectorization(eng)
    return ({"encoder_inputs": kor, "decoder_inputs": eng[:, :-1],}, eng[:, 1:])


def make_dataset(pairs):
    kor_texts, eng_texts = zip(*pairs)
    kor_texts = list(kor_texts)
    eng_texts = list(eng_texts)
    dataset = tf.data.Dataset.from_tensor_slices((kor_texts, eng_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [45]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (64, 20)
inputs["decoder_inputs"].shape: (64, 20)
targets.shape: (64, 20)


In [46]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)

In [47]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

In [48]:
epochs = 1  # This should be at least 30 for convergence

transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)
transformer.fit(train_ds, epochs=epochs, validation_data=val_ds)

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 positional_embedding_4 (Positi  (None, None, 256)   3845120     ['encoder_inputs[0][0]']         
 onalEmbedding)                                                                                   
                                                                                                  
 decoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 transformer_encoder_2 (Transfo  (None, None, 256)   3155456     ['positional_embedding_

InvalidArgumentError: Graph execution error:

Detected at node 'transformer/transformer_encoder_2/multi_head_attention_6/softmax/add' defined at (most recent call last):
    File "C:\Users\DELL\anaconda3\envs\mytorch\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\DELL\anaconda3\envs\mytorch\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\DELL\anaconda3\envs\mytorch\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\DELL\anaconda3\envs\mytorch\lib\site-packages\traitlets\config\application.py", line 982, in launch_instance
      app.start()
    File "C:\Users\DELL\anaconda3\envs\mytorch\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\DELL\anaconda3\envs\mytorch\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_foreve